In [7]:
import os
import math
from pyomo.environ import *
import pandas as pd
import time

In [8]:
# configuracion parametros para el solver y el modelo
solver_name="glpk"
gap=0.02
Tiempo_maximo_corrida_cluster=300 # en segundos
personas_esperadas_cluster=20
default_P_Tmax_PD=1
factor_HQ=2


# carpeta con todos los inputs
directory = 'C:/Users/jburitica99/inputs_sura'


# nombres de las hojas
Hojas_vector=['P_PQ(p,q)','P_CQ(c,q)','P_Di_PC(p,c)','P_Di_PDH(p,d,h)','P_Di_CDH(c,d,h)','P_Hmax_PD(p,d)','P_Hmin_PD(p,d)','P_Di_TH(t,h)','P_Hcon_P(p)','P_Tmax_PD(p,d)']
Hojas_vector_1=['P_Sim_PDD_(p,d,d)','P_Sim_PCDD_(p,c,d,d)','P_Fat10_PDHDH_(p,d,h,d,h)','Dias_semana']



In [9]:
for filename in os.listdir(directory):
    input_file_name=directory+'/'+filename

    # lectura datos de entrada
    for hoja in Hojas_vector:
        df = pd.read_excel(input_file_name,sheet_name=hoja)
        nfil=df.shape[0]
        ncol=df.shape[1]
        header=[[(i) for i in df.columns]]
        rows=[[(df.iloc[i,j]) for j in range(0,df.shape[1])] for i in range(0,df.shape[0])]
        filas=header+rows


        dic_bd={}

        if ncol>2:
            for i in range(1,nfil+1):
                index_tuples=()
                for j in range(ncol-1):
                    index_tuples=index_tuples+(filas[i][j],)
                dic_bd[index_tuples] = filas[i][ncol-1]
        else:
            for i in range(1,nfil+1):
                dic_bd[filas[i][0]]=filas[i][1]

        # Parametros para el modelo de clusterizacion -----------------------------------------------
        if hoja=='P_PQ(p,q)':
            dic_P_PQ=dic_bd.copy()

        elif hoja=='P_CQ(c,q)':
            dic_P_CQ=dic_bd.copy()
            qualificaciones=list(dict.fromkeys([filas[i][1] for i in range(1,nfil+1)]))

        # Parametros para el modelo de optimizacion de espacios fisicos ----------------------------- 
        elif hoja=='P_Di_PC(p,c)':
            dic_P_Di_PC=dic_bd.copy()
            lis_P_Di_PC=list(dic_P_Di_PC.keys())
            personas=list(dict.fromkeys([filas[i][0] for i in range(1,nfil+1)]))
            consultorios=list(dict.fromkeys([filas[i][1] for i in range(1,nfil+1)]))        

        elif hoja=='P_Di_PDH(p,d,h)':
            dic_P_Di_PDH=dic_bd.copy()


        elif hoja=='P_Di_CDH(c,d,h)':
            dic_P_Di_CDH=dic_bd.copy()

        elif hoja=='P_Hmax_PD(p,d)':
            dic_P_Hmax_PD=dic_bd.copy()

        elif hoja=='P_Hmin_PD(p,d)':
            dic_P_Hmin_PD=dic_bd.copy()

        elif hoja=='P_Di_TH(t,h)':
            dic_P_Di_TH=dic_bd.copy()
            turnos=list(dict.fromkeys([filas[i][0] for i in range(1,nfil+1)]))
            horas=list(dict.fromkeys([filas[i][1] for i in range(1,nfil+1)]))

        elif hoja=='P_Hcon_P(p)':
            dic_P_Hcon_P=dic_bd.copy()

        elif hoja=='P_Tmax_PD(p,d)':
            dic_P_Tmax_PD=dic_bd.copy()



    for hoja_1 in Hojas_vector_1:
        df = pd.read_excel(input_file_name,sheet_name=hoja_1)
        nfil=df.shape[0]
        ncol=df.shape[1]

        if hoja_1=='P_Sim_PDD_(p,d,d)':
            simetria_1=[[(df.iloc[i,j]) for j in range(0,df.shape[1])] for i in range(0,df.shape[0])]

        elif hoja_1=='P_Sim_PCDD_(p,c,d,d)':
            simetria_2=[[(df.iloc[i,j]) for j in range(0,df.shape[1])] for i in range(0,df.shape[0])]

        elif hoja_1=='P_Fat10_PDHDH_(p,d,h,d,h)':
            fatiga_10h=[[(df.iloc[i,j]) for j in range(0,df.shape[1])] for i in range(0,df.shape[0])]

        elif hoja_1=='Dias_semana':
            dias_semana=[[(df.iloc[i,j]) for j in range(0,df.shape[1])] for i in range(0,df.shape[0])]
            semanas=list(dict.fromkeys([df.iloc[i,0] for i in range(0,df.shape[0])]))
            dias=list(dict.fromkeys([df.iloc[i,1] for i in range(0,df.shape[0])]))


    # Calular el numero de grupos iniciales a crear para la sede
    numero_grupos_arranque=math.ceil(len(personas)/personas_esperadas_cluster)
    grupos_resueltos=0
    P_NG=numero_grupos_arranque

    # calcular horas maximas disponible por consultorio en todo el periodo
    P_HAC=(max(horas)-min(horas))*len(dias)



    # iniciar a resolver los subgrupos hasta que todos sean resueltos
    while grupos_resueltos< P_NG:

        # Resolver modelo de clusterizacion si numero de grupos <1
        grupos=[i1 for i1 in range(1,P_NG+1)]

        # guardar resultados del modelo de clusterizacion
        dic_res_1={}
        dic_res_2={}

        if P_NG>1:

            P_PMG=len(personas)/P_NG
            P_HAMG=(len(consultorios)/P_NG)*P_HAC
            P_HDMG=sum(dic_P_Hcon_P.values())/P_NG

            #----------------------------------------------------------------------------------------------------------------

            ###################################-----------------------#######################################################
            # 1.Resolver el modelo de clusterizacion

            # formular y resolver modelo de programacion entera mixta para los espacios fisicos
            model = ConcreteModel()

            # declaracion de sets simples -----------------------------------------------------
            model.set_personas=Set(initialize=personas, doc='personas')
            model.set_consultorios=Set(initialize=consultorios, doc='consultorios')
            model.set_qualificaciones=Set(initialize=qualificaciones, doc='qualificaciones')
            model.set_grupos=Set(initialize=grupos, doc='grupos') 

            # Declaracion de parametros ------------------------------------------------------------------------
            model.par_P_HD_P = Param(model.set_personas,initialize=dic_P_Hcon_P, default=0, doc='horas contratadas pers_sem')
            model.par_P_PQ = Param(model.set_personas,model.set_qualificaciones,initialize=dic_P_PQ, default=0, doc='relacion persona-qualificacion')
            model.par_P_CQ = Param(model.set_consultorios,model.set_qualificaciones,initialize=dic_P_CQ, default=0, doc='relacion consul-qualificacion')

            # Declaracion de variables -------------------------------------------------------------------------------------------------------------------------------------------
            model.var_V_A_PG= Var(model.set_personas, model.set_grupos, within=Binary, bounds=(0.0,None), doc='asignacion persona a grupo')
            model.var_V_A_CG= Var(model.set_consultorios, model.set_grupos, within=Binary, bounds=(0.0,None), doc='asignacion consultorio a grupo')


            # Declaracion de variables auxiliares --------------------------------------------------------------------------------------------------------------------
            model.var_V_HA_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_HA_QG = Var(model.set_qualificaciones,model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_HD_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_HD_QG = Var(model.set_qualificaciones,model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_TP_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))

            # Declaracion de otras variables auxiliares --------------------------------------------------------------------------------------------------------------------
            model.var_V_DesPosP_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_DesNegP_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_DesPosHD_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_DesNegHD_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_DesPosHA_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_DesNegHA_G = Var(model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))    
            model.var_V_HDef_QG = Var(model.set_qualificaciones,model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))
            model.var_V_HHol_QG = Var(model.set_qualificaciones,model.set_grupos, within=NonNegativeReals, bounds=(0.0,None))


            # Restricciones -----------------------------------------------------------------------------------------------------

            # R1: asignar todas las personas a solo un grupo ------------------------------------------------
            model.con_R1 = ConstraintList(doc='R1: asignar todas las personas a solo un grupo')
            for p in model.set_personas:
                model.con_R1.add(sum(model.var_V_A_PG[p,g] for g in model.set_grupos)==1)

            # R2: asignar todos los consultorios a solo un grupo ------------------------------------------------
            model.con_R2 = ConstraintList(doc='R2: asignar todos los consultorios a solo un grupo')
            for c in model.set_consultorios:
                model.con_R2.add(sum(model.var_V_A_CG[c,g] for g in model.set_grupos)==1)

            # R3: calcular horas asignadas a grupo ------------------------------------------------
            model.con_R3 = ConstraintList(doc='R3: calcular horas asignadas a grupo')
            for g in model.set_grupos:
                model.con_R3.add(sum(model.var_V_A_CG[c,g]*P_HAC for c in model.set_consultorios)==model.var_V_HA_G[g])    

            # R4: calcular horas asignadas por qualificacion y grupo ------------------------------------------------
            model.con_R4 = ConstraintList(doc='R4: calcular horas asignadas por qualificacion y grupo')
            for q in model.set_qualificaciones:
                for g in model.set_grupos:
                    model.con_R4.add(sum(model.var_V_A_CG[c,g]*P_HAC*model.par_P_CQ[c,q] for c in model.set_consultorios)==model.var_V_HA_QG[q,g])    

            # R5: calcular horas demandadas por grupo ------------------------------------------------
            model.con_R5 = ConstraintList(doc='R5: calcular horas demandadas por grupo')
            for g in model.set_grupos:
                model.con_R5.add(sum(model.var_V_A_PG[p,g]*model.par_P_HD_P[p] for p in model.set_personas)==model.var_V_HD_G[g])


            # R6: calcular horas demandas por qualificacion y grupo ------------------------------------------------
            model.con_R6 = ConstraintList(doc='R6: calcular horas demandas por qualificacion y grupo')
            for q in model.set_qualificaciones:
                for g in model.set_grupos:
                    model.con_R6.add(sum(model.var_V_A_PG[p,g]*model.par_P_HD_P[p]*model.par_P_PQ[p,q] for p in model.set_personas)==model.var_V_HD_QG[q,g])

            # R7: calcular total personas por grupo ------------------------------------------------
            model.con_R7 = ConstraintList(doc='R7: calcular total personas por grupo')
            for g in model.set_grupos:
                model.con_R7.add(sum(model.var_V_A_PG[p,g] for p in model.set_personas)==model.var_V_TP_G[g])

            # R8: calcular desviaciones de personas por grupo ------------------------------------------------
            model.con_R8 = ConstraintList(doc='R8: calcular desviaciones de personas por grupo')
            for g in model.set_grupos:
                model.con_R8.add(model.var_V_TP_G[g]-P_PMG==model.var_V_DesPosP_G[g]-model.var_V_DesNegP_G[g])

            # R9: calcular desviaciones de horas demandadas por grupo ------------------------------------------------
            model.con_R9 = ConstraintList(doc='R9: calcular desviaciones de horas demandadas por grupo')
            for g in model.set_grupos:
                model.con_R9.add(model.var_V_HD_G[g]-P_HDMG==model.var_V_DesPosHD_G[g]-model.var_V_DesNegHD_G[g])

            # R10: calcular desviaciones de horas asignadas por grupo ------------------------------------------------
            model.con_R10 = ConstraintList(doc='R10: calcular desviaciones de horas asignadas por grupo')
            for g in model.set_grupos:
                model.con_R10.add(model.var_V_HA_G[g]-P_HAMG==model.var_V_DesPosHA_G[g]-model.var_V_DesNegHA_G[g])

            # R11: calcular horas de deficit o de holgura por qualificacion y grupo ------------------------------------------------
            model.con_R11 = ConstraintList(doc='R11: calcular horas de deficit o de holgura por qualificacion y grupo')
            for q in model.set_qualificaciones:
                for g in model.set_grupos:
                    model.con_R11.add((factor_HQ*model.var_V_HD_QG[q,g])-model.var_V_HA_QG[q,g]==model.var_V_HDef_QG[q,g]-model.var_V_HHol_QG[q,g])

            # funcion objetivo ----------------------------------------------------------------------------------------------------
            def objective_rule(model):     
                return (500*sum(model.var_V_DesPosP_G[g]+model.var_V_DesNegP_G[g] for g in model.set_grupos)+
                        10*sum(model.var_V_DesPosHD_G[g]+model.var_V_DesNegHD_G[g] for g in model.set_grupos)+
                        10*sum(model.var_V_DesPosHA_G[g]+model.var_V_DesNegHA_G[g] for g in model.set_grupos)+
                        sum(model.var_V_HDef_QG[q,g] for q in model.set_qualificaciones for g in model.set_grupos)
                       )
            model.objective = Objective(rule=objective_rule, sense=minimize, doc='Objective function')

            # resolver el modelo --------------------------------------------------------------------------------------------------

            instance = model 
            solver = SolverFactory(solver_name)

            if solver_name=="glpk":
                solver.options["mipgap"] = gap
                solver.options["tmlim"] = Tiempo_maximo_corrida_cluster

            elif solver_name=="cbc":
                solver.options['seconds'] = Tiempo_maximo_corrida_cluster
                solver.options["RatioGap"] = gap
                #solver.options["MaxSolutions"] = 1
                #solver.options['integerTolerance'] = 1E-4
            print(' ')   
            print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
            print('resolviendo el modelo de clusterizacion---------------------------------')
            print(' ')

            resultados = solver.solve(model)
            resultados.write()    
            print('resuelto el modelo de clusterizacion---------------------------------')
            print(' ')
            print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
            print(' ')


            for g in model.set_grupos:
                for p in model.set_personas:
                    dic_res_1[p,g]=model.var_V_A_PG[p,g].value
                for c in model.set_consultorios:
                    dic_res_2[c,g]=model.var_V_A_CG[c,g].value


            # 1.fin modelo de clusterizacion
            ###################################-----------------------#######################################################

        # si hay un solo cluster por resolver
        else:
            for g in grupos:
                for p in personas:
                    dic_res_1[p,g]=1
                for c in consultorios:
                    dic_res_2[c,g]=1        

        # fin clusterizacion
        #----------------------------------------------------------------------------------------------------------------


        # Resolver Modelo de optimizacion para cada cluster-grupo---------------------------------------------------

        # guardar encabezados de los resultados del modelo espacios fisicos--------------------------------------------
        resultados1=[['persona','consultorio','turno','dia','valor']]
        resultados2=[['persona','horas no programadas']]
        resultados3=[['persona','dia','esta activa (1:si)']]
        resultados4=[['consultorio','dia','hora','esta activo (1:si)']]

        #for clus in range(4,5):
        #for clus in range(1,P_NG+1):
        clus=1
        aumentar_cluster=0
        while (clus<=P_NG and aumentar_cluster<1):

            print(' ')   
            print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
            print('resolviendo cluster '+ str(clus) + ' de ' + str(P_NG))

            ###################################-----------------------##########################

            #crear listas indexedas por cluster
            personas_cluster=[]
            consultorios_cluster=[]
            lis_P_Di_PC_cluster=[]

            for p in personas:
                if dic_res_1[p,clus]==1:
                    personas_cluster.append(p)
            for c in consultorios:
                if dic_res_2[c,clus]==1:
                    consultorios_cluster.append(c)
            for p_c in lis_P_Di_PC:
                if p_c[0] in personas_cluster and p_c[1] in consultorios_cluster:
                    lis_P_Di_PC_cluster.append(p_c)

            dic_P_Di_PDH_cluster=dic_P_Di_PDH.copy()
            for i in dic_P_Di_PDH:
                if i[0] not in personas_cluster:
                    del dic_P_Di_PDH_cluster[i]
            #-------------------
            dic_P_Hmax_PD_cluster=dic_P_Hmax_PD.copy()
            for i in dic_P_Hmax_PD:
                if i[0] not in personas_cluster:
                    del dic_P_Hmax_PD_cluster[i]
            #-------------------
            dic_P_Hmin_PD_cluster=dic_P_Hmin_PD.copy()
            for i in dic_P_Hmin_PD:
                if i[0] not in personas_cluster:
                    del dic_P_Hmin_PD_cluster[i]
            #-------------------
            dic_P_Hcon_P_cluster=dic_P_Hcon_P.copy()
            for i in dic_P_Hcon_P:
                if i not in personas_cluster:
                    del dic_P_Hcon_P_cluster[i]
            #-------------------
            dic_P_Tmax_PD_cluster=dic_P_Tmax_PD.copy()
            for i in dic_P_Tmax_PD:
                if i[0] not in personas_cluster:
                    del dic_P_Tmax_PD_cluster[i]

            #-------------------
            dic_P_Di_CDH_cluster=dic_P_Di_CDH.copy()
            for i in dic_P_Di_CDH:
                if i[0] not in consultorios_cluster:
                    del dic_P_Di_CDH_cluster[i]


            #-------------------
            simetria_1_cluster=[]
            for i in simetria_1:
                if i[0] in personas_cluster:
                    simetria_1_cluster.append(i) 

            #-------------------
            simetria_2_cluster=[]
            for i in simetria_2:
                if i[0] in personas_cluster:
                    simetria_2_cluster.append(i) 

            #-------------------
            fatiga_10h_cluster=[]
            for i in fatiga_10h:
                if i[0] in personas_cluster:
                    fatiga_10h_cluster.append(i) 
            #-------------------

            # 2. fin estandarización parametros de cluster
            ###################################-----------------------##########################

            #----------------------------------------------------------------------------------------------------------------

            #################################-------------------#########################
            # 3. modelo de optimzacion para el grupo ---------------------------------

            model = ConcreteModel()


            # declaracion de sets simples --------------------------------------------------------------------------
            model.set_personas=Set(initialize=personas_cluster, doc='personas')
            model.set_consultorios=Set(initialize=consultorios_cluster, doc='consultorios')
            model.set_dias=Set(initialize=dias, doc='dias')
            model.set_horas=Set(initialize=horas, doc='horas')
            model.set_turnos=Set(initialize=turnos, doc='turnos')

            # declaracion de sets conjuntos --------------------------------------------------------------------------
            model.set_personas_consultorios=Set(initialize=lis_P_Di_PC_cluster, doc='personas-consultorios')

            # Declaracion de parametros ------------------------------------------------------------------------
            #model.par_P_Di_PC = Param(model.set_personas_consultorios,initialize=dic_P_Di_PC_cluster, default=0, doc='combinaciones factibles pers_cons')
            model.par_P_Di_PDH = Param(model.set_personas, model.set_dias,model.set_horas,initialize=dic_P_Di_PDH_cluster, default=0, doc='disponibilidad pers_dias_horas')
            model.par_P_Di_CDH = Param(model.set_consultorios, model.set_dias,model.set_horas,initialize=dic_P_Di_CDH_cluster, default=1, doc='disponibilidad cons_dias_horas')
            model.par_P_Hmax_PD = Param(model.set_personas, model.set_dias,initialize=dic_P_Hmax_PD_cluster, default=12, doc='horas max perso_dia')
            model.par_P_Hmin_PD = Param(model.set_personas, model.set_dias,initialize=dic_P_Hmin_PD_cluster, default=0, doc='horas min perso_dia')
            model.par_P_Di_TH = Param(model.set_turnos, model.set_horas,initialize=dic_P_Di_TH, default=0, doc='disponibilidad turno_hora')
            model.par_P_Hcon_P = Param(model.set_personas,initialize=dic_P_Hcon_P_cluster, default=0, doc='horas contratadas pers_sem')
            model.par_P_Tmax_PD = Param(model.set_personas,model.set_dias,initialize=dic_P_Tmax_PD_cluster, default=default_P_Tmax_PD, doc='maximas comb turno_cons por  pers_dia')


            # Declaracion de variables -------------------------------------------------------------------------------------------------------------------------------------------
            model.var_V_A_PTCD = Var(model.set_personas_consultorios, model.set_turnos, model.set_dias, within=Binary, bounds=(0.0,None), doc='asignacion persona_turno_cons_dia')

            # Declaracion de variables auxiliares --------------------------------------------------------------------------------------------------------------------
            model.var_V_HD_P = Var(model.set_personas, within=NonNegativeReals, bounds=(0.0,None), doc='horas deficit persona_semana')
            model.var_V_A_PD = Var(model.set_personas, model.set_dias, within=Binary, bounds=(0.0,None), doc='asignacion persona_dia')


            # Restricciones -----------------------------------------------------------------------------------------------------

            print('----------------------------------------------------------')
            print('resolviendo cluster ' + str(clus))
            print('agregando restricciones')
            print(time.strftime("%H:%M:%S"))

            # Restriccion 1: R1 - maximo una combinacion turno-consultorio por persona, dia y hora si la persona esta disponible-------------------------------------------------
            model.con_R1 = ConstraintList(doc='maximo una combinacion turno-consultorio por persona, dia y hora')

            # funcion para calcular consultorios indexados a personas
            def consul_inde_p(p1):
                vector_consul_inde_p=[]
                for i1 in lis_P_Di_PC_cluster:
                    if i1[0]==p1:
                        vector_consul_inde_p.append(i1[1])
                return vector_consul_inde_p
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                if len(set_consultorios_p)>0:
                    for d in model.set_dias:
                        for h in model.set_horas:
                            model.con_R1.add(
                                sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                    for t in model.set_turnos for c in set_consultorios_p) 
                                <=
                                model.par_P_Di_PDH[p,d,h]
                            )
            print('agregada R1')
            print(time.strftime("%H:%M:%S"))

            # Restriccion 2: R2 - maximo una combinacion persona-turno por consultorio, dia y hora si consultorio esta disponible-------------------------------------------------
            model.con_R2 = ConstraintList(doc='maximo una combinacion persona-turno por consultorio, dia y hora')

            # funcion para calcular personas indexadas por consultorio
            def personas_inde_c(c1):
                vector_personas_inde_c=[]
                for i1 in lis_P_Di_PC_cluster:
                    if i1[1]==c1:
                        vector_personas_inde_c.append(i1[0])
                return vector_personas_inde_c
            for c in model.set_consultorios:
                set_personas_c=personas_inde_c(c)
                if len(set_personas_c)>0:
                    for d in model.set_dias:
                        for h in model.set_horas:
                            model.con_R2.add(
                                sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                    for t in model.set_turnos for p in set_personas_c) 
                                <=
                                model.par_P_Di_CDH[c,d,h]
                            )
            print('agregada R2')
            print(time.strftime("%H:%M:%S"))

            # Restriccion 4: R4 - No asignar mas horas de las permitidas por persona y dia - si la persona se activa -------------------------------------------------
            model.con_R4 = ConstraintList(doc='No asignar mas horas de las permitidas por persona y dia')
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                if len(set_consultorios_p)>0:
                    for d in model.set_dias:
                        if model.par_P_Tmax_PD[p,d]>0:
                            model.con_R4.add(
                                    sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                        for t in model.set_turnos for c in set_consultorios_p for h in model.set_horas) 
                                    <=
                                    model.par_P_Hmax_PD[p,d]*model.var_V_A_PD[p,d]
                                )
            print('agregada R4')
            print(time.strftime("%H:%M:%S"))


            # Restriccion 5: R5 - No asignar menos horas de las permitidas por persona y dia - si la persona se activa -------------------------------------------------
            model.con_R5 = ConstraintList(doc='No asignar menos horas de las permitidas por persona y dia')
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                if len(set_consultorios_p)>0:
                    for d in model.set_dias:
                        if model.par_P_Hmin_PD[p,d]>0:
                            model.con_R5.add(
                                    sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                        for t in model.set_turnos for c in set_consultorios_p for h in model.set_horas) 
                                    >=
                                    model.par_P_Hmin_PD[p,d]*model.var_V_A_PD[p,d]
                                )
            print('agregada R5')
            print(time.strftime("%H:%M:%S"))

            # Restriccion 6: R6 - No asignar mas combinaciones turnos-consultorios permitidos por persona y dia si la persona se activa-------------------------------------------------
            model.con_R6 = ConstraintList(doc='No asignar mas combinaciones turnos-consultorios permitidos por persona')
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                if len(set_consultorios_p)>0:
                    for d in model.set_dias:
                        model.con_R6.add(
                                sum(model.var_V_A_PTCD[p,c,t,d]
                                    for t in model.set_turnos for c in set_consultorios_p) 
                                <=
                                model.par_P_Tmax_PD[p,d]*model.var_V_A_PD[p,d]
                            )
            print('agregada R6')
            print(time.strftime("%H:%M:%S"))


            # Restriccion 7: R7 - Calcular horas deficit por persona a la semana ---------------------------------------
            model.con_R7 = ConstraintList(doc='Calcular horas deficit por persona a la semana')
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                if len(set_consultorios_p)>0:
                    model.con_R7.add(
                            sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                for t in model.set_turnos for c in set_consultorios_p 
                                for h in model.set_horas for d in model.set_dias)
                            + model.var_V_HD_P[p]
                            ==
                            model.par_P_Hcon_P[p]
                        )
            print('agregada R7')
            print(time.strftime("%H:%M:%S"))


            # Restriccion 8: R8 - simetria 1: simetria a nivel persona, dia1, dia2 -----------------------------------------
            # garantizar que la persona p, en el dia1 y dia2, trabaje las mismas horas
            if len(simetria_1_cluster)>0:
                model.con_R8 = ConstraintList(doc='simetria 1: simetria a nivel persona, dia1, dia2')
                for sim_1 in simetria_1_cluster:
                    p=sim_1[0]
                    dia1=sim_1[1]
                    dia2=sim_1[2]
                    set_consultorios_p=consul_inde_p(p)
                    if len(set_consultorios_p)>0:
                        for t in model.set_turnos:
                            model.con_R8.add(sum(model.var_V_A_PTCD[p,c,t,dia1] for c in set_consultorios_p)==
                                             sum(model.var_V_A_PTCD[p,c,t,dia2] for c in set_consultorios_p)
                                            )
                print('agregada R8')
                print(time.strftime("%H:%M:%S"))
                #model.con_R8.pprint()


            # Restriccion 9: R9 - simetria 2: simetria a nivel persona,consultorio, dia1, dia2 -----------------------------------------
            # garantizar que la persona p, en el dia1 y dia2, trabaje las mismas horas y en el mismo consultorio
            if len(simetria_2_cluster)>0:
                model.con_R9 = ConstraintList(doc='simetria 2: simetria a nivel persona,consultorio, dia1, dia2')
                for sim_2 in simetria_2_cluster:
                    p=sim_2[0]
                    dia1=sim_2[1]
                    dia1=sim_2[2]
                    set_consultorios_p=consul_inde_p(p)
                    if len(set_consultorios_p)>0:
                        for c in set_consultorios_p:
                            for t in model.set_turnos:
                                model.con_R9.add(model.var_V_A_PTCD[p,c,t,dia1]==
                                                 model.var_V_A_PTCD[p,c,t,dia2]
                                                )
                print('agregada R9')
                print(time.strftime("%H:%M:%S"))
                #model.con_R9.pprint()      


            # Restriccion 10: R10 - fatiga 10 horas: minimo 10 horas de descanso entre dias consecutivos---------------------
            # garantizar que toda pesona, descanse al menos 10 horas en dias consecutivos

            # funcion para calcular turnos indexados a horas
            def turnos_inde_h(h1):
                vector_turnos_inde_h=[]
                for i1 in dic_P_Di_TH:
                    if i1[1]==h1:
                        vector_turnos_inde_h.append(i1[0])
                return vector_turnos_inde_h
            if len(fatiga_10h_cluster)>0:
                model.con_R10 = ConstraintList(doc='fatiga 10 horas: minimo 10 horas de descanso entre dias consecutivos')
                for fat_10 in fatiga_10h_cluster:
                    p=fat_10[0]
                    dia1=fat_10[1]
                    hora1=fat_10[2]
                    dia2=fat_10[3]
                    hora2=fat_10[4]                
                    set_turnos_dis_h1=turnos_inde_h(hora1)
                    set_turnos_dis_h2=turnos_inde_h(hora2)
                    set_consultorios_p=consul_inde_p(p)
                    if len(set_consultorios_p)>0:
                        model.con_R10.add(sum(model.var_V_A_PTCD[p,c1,t1,dia1] for c1 in set_consultorios_p for t1 in set_turnos_dis_h1)+
                                          sum(model.var_V_A_PTCD[p,c2,t2,dia2] for c2 in set_consultorios_p for t2 in set_turnos_dis_h2)<=1
                                         )

                print('agregada R10')
                print(time.strftime("%H:%M:%S"))
                #model.con_R10.pprint() 

            # Restriccion 11: R11 - fatiga semana: maximo 60 horas de trabajo por semana ---------------------
            # garantizar que toda pesona, trabaje maximo 60 horas a la semana

            if len(semanas)>1:
                # funcion para calcular dias indexados a semana
                def dias_inde_sem(s1):
                    vector_dias_inde_s=[]
                    for i1 in dias_semana:
                        if i1[0]==s1:
                            vector_dias_inde_s.append(i1[1])
                    return vector_dias_inde_s

                model.con_R11 = ConstraintList(doc='fatiga semana: maximo 60 horas de trabajo por semana')
                for p in model.set_personas:
                    set_consultorios_p=consul_inde_p(p)
                    if len(set_consultorios_p)>0:
                        for s in semanas:
                            set_dias_semana_s=dias_inde_sem(s)
                            model.con_R11.add(
                                sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h] 
                                    for t in model.set_turnos for c in set_consultorios_p
                                    for h in model.set_horas for d in set_dias_semana_s)
                                <=60
                            )        
                print('agregada R11')
                print(time.strftime("%H:%M:%S"))                    

            # ---------------------- fin restricciones ---------------------------------------------------------               

            # funcion objetivo ----------------------------------------------------------------------------------------------------
            def objective_rule(model):     
                return (sum(model.var_V_HD_P[p] for p in model.set_personas))
            model.objective = Objective(rule=objective_rule, sense=minimize, doc='Objective function')

            # Solucion del modelo -------------------------------------------------------------------------------------------------
            instance = model 
            solver = SolverFactory(solver_name)

            if solver_name=="glpk":
                solver.options["mipgap"] = gap
                solver.options["tmlim"] = Tiempo_maximo_corrida_cluster

            elif solver_name=="cbc":
                solver.options['seconds'] = Tiempo_maximo_corrida_cluster
                solver.options["RatioGap"] = gap
                #solver.options["MaxSolutions"] = 1
                #solver.options['integerTolerance'] = 1E-4


            print('resolviendo el modelo')
            resultados = solver.solve(model)



            # 3. fin modelo de optimizacion para el grupo -----------------------------
            #################################-------------------#########################



            # si se llega a la solucion factible o la optima

            if (resultados.solver.termination_condition=='optimal' or resultados.solver.termination_condition=='feasible'):
                clus=clus+1
                print(' ')   
                print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')            
                print('modelo resuelto----------------------------------------')
                resultados.write()  

                print(time.strftime("%H:%M:%S"))
                grupos_resueltos=grupos_resueltos+1
                # guardar resultados--------------------------------------------
                for p_c in model.set_personas_consultorios:
                    for t in model.set_turnos:
                        for d in model.set_dias:
                            if (model.var_V_A_PTCD[p_c,t,d].value>0):
                                resultados1.append([p_c[0],p_c[1],t,d,model.var_V_A_PTCD[p_c,t,d].value])
                                #print(str((p_c,t,d))+str( ' = ')+str(model.var_V_A_PTCD[p_c,t,d].value))


                for p in model.set_personas:
                    resultados2.append([p,model.var_V_HD_P[p].value])
                    #print(p,model.var_V_HD_P[p].value)
                    for d in model.set_dias:
                        resultados3.append([p,d,model.var_V_A_PD[p,d].value])

                for c in model.set_consultorios:
                    set_personas_c=personas_inde_c(c)
                    for d in model.set_dias:
                        for h in model.set_horas:
                            set_turnos_dis_h=turnos_inde_h(h)
                            cons_act=sum(model.var_V_A_PTCD[p,c,t,d].value for p in set_personas_c for t in set_turnos_dis_h)
                            resultados4.append([c,d,h,cons_act])

            else:
                aumentar_cluster=1 # se para la solucion de los cluster con el numero actual y se incrementa en 1 porque al menos 1 cluster fallo

        #----------------------------------------------------------------------------------------------------------------

        # preguntar si se resovieron todos los grupos
        # si no se resolvieron incrementar un grupo (crear un cluster mas) e inicializar la variable de grupos_resueltos
            #correr nuevamente modelo de clusterizacion y de optimizacion por cluster
        #grupos_resueltos=4    
        if grupos_resueltos<P_NG:
            P_NG=P_NG+1
            grupos_resueltos=0

    # guardar resultados clusters primarios -----------------------------------------------------------------------------

    data1=pd.DataFrame.from_records(resultados1[1:], columns=resultados1[0])
    data2=pd.DataFrame.from_records(resultados2[1:], columns=resultados2[0])
    data3=pd.DataFrame.from_records(resultados3[1:], columns=resultados3[0])
    data4=pd.DataFrame.from_records(resultados4[1:], columns=resultados4[0])



    # correr cluster residual si hay mas de un cluster
    if P_NG>1:
        # calcular listas y parametros para el cluster residual
        personas_cluster_residual=data2[data2['horas no programadas']>0]['persona'].tolist()
        disp_personas_dia_cluster_residual=data3[data3['esta activa (1:si)']==0][['persona','dia']].to_numpy().tolist()
        dic_P_Di_PDH_cluster_residual=dic_P_Di_PDH.copy()

        # desactivar personas en los dias que ya han sido programados
        for i in dic_P_Di_PDH:
            if (([i[0],i[1]] not in disp_personas_dia_cluster_residual) or (i[0] not in personas_cluster_residual)):
                del dic_P_Di_PDH_cluster_residual[i]

        # desactivar consultorios en las horas que ya han sido programdos        
        nueva_indisponib_consul_cluster_residual=data4[data4['esta activo (1:si)']==1].to_numpy().tolist()
        dic_P_Di_CDH_cluster_residual=dic_P_Di_CDH.copy()
        for i in nueva_indisponib_consul_cluster_residual:
            dic_P_Di_CDH_cluster_residual[i[0],i[1],i[2]]=0

        # calcular las horas pendientes por programar por persona
        horas_contr_clus_residual=data2[data2['horas no programadas']>0].to_numpy().tolist()
        dic_P_Hcon_P_cluster_residual={}
        for i in horas_contr_clus_residual:
            dic_P_Hcon_P_cluster_residual[int(i[0])]=i[1]

        ###################################-----------------------##########################

        #crear listas indexedas pora clsuter residual
        personas_cluster=personas_cluster_residual
        consultorios_cluster=consultorios
        lis_P_Di_PC_cluster=[]

        for p_c in lis_P_Di_PC:
            if p_c[0] in personas_cluster and p_c[1] in consultorios_cluster:
                lis_P_Di_PC_cluster.append(p_c)

        dic_P_Di_PDH_cluster=dic_P_Di_PDH_cluster_residual
        #-------------------
        dic_P_Hmax_PD_cluster=dic_P_Hmax_PD.copy()
        for i in dic_P_Hmax_PD:
            if i[0] not in personas_cluster:
                del dic_P_Hmax_PD_cluster[i]
        #-------------------
        dic_P_Hmin_PD_cluster=dic_P_Hmin_PD.copy()
        for i in dic_P_Hmin_PD:
            if i[0] not in personas_cluster:
                del dic_P_Hmin_PD_cluster[i]
        #-------------------
        dic_P_Hcon_P_cluster=dic_P_Hcon_P_cluster_residual
        #-------------------
        dic_P_Tmax_PD_cluster=dic_P_Tmax_PD.copy()
        for i in dic_P_Tmax_PD:
            if i[0] not in personas_cluster:
                del dic_P_Tmax_PD_cluster[i]

        #-------------------
        dic_P_Di_CDH_cluster=dic_P_Di_CDH_cluster_residual

        #-------------------
        simetria_1_cluster=[]
        for i in simetria_1:
            if i[0] in personas_cluster:
                simetria_1_cluster.append(i) 

        #-------------------
        simetria_2_cluster=[]
        for i in simetria_2:
            if i[0] in personas_cluster:
                simetria_2_cluster.append(i) 

        #-------------------
        fatiga_10h_cluster=[]
        for i in fatiga_10h:
            if i[0] in personas_cluster:
                fatiga_10h_cluster.append(i) 
        #-------------------

        # 2. fin estandarización parametros de cluster
        ###################################-----------------------##########################

        #----------------------------------------------------------------------------------------------------------------

        #################################-------------------#########################
        # 3. modelo de optimzacion para el cluster residual ---------------------------------

        model = ConcreteModel()


        # declaracion de sets simples --------------------------------------------------------------------------
        model.set_personas=Set(initialize=personas_cluster, doc='personas')
        model.set_consultorios=Set(initialize=consultorios_cluster, doc='consultorios')
        model.set_dias=Set(initialize=dias, doc='dias')
        model.set_horas=Set(initialize=horas, doc='horas')
        model.set_turnos=Set(initialize=turnos, doc='turnos')

        # declaracion de sets conjuntos --------------------------------------------------------------------------
        model.set_personas_consultorios=Set(initialize=lis_P_Di_PC_cluster, doc='personas-consultorios')

        # Declaracion de parametros ------------------------------------------------------------------------
        #model.par_P_Di_PC = Param(model.set_personas_consultorios,initialize=dic_P_Di_PC_cluster, default=0, doc='combinaciones factibles pers_cons')
        model.par_P_Di_PDH = Param(model.set_personas, model.set_dias,model.set_horas,initialize=dic_P_Di_PDH_cluster, default=0, doc='disponibilidad pers_dias_horas')
        model.par_P_Di_CDH = Param(model.set_consultorios, model.set_dias,model.set_horas,initialize=dic_P_Di_CDH_cluster, default=1, doc='disponibilidad cons_dias_horas')
        model.par_P_Hmax_PD = Param(model.set_personas, model.set_dias,initialize=dic_P_Hmax_PD_cluster, default=12, doc='horas max perso_dia')
        model.par_P_Hmin_PD = Param(model.set_personas, model.set_dias,initialize=dic_P_Hmin_PD_cluster, default=0, doc='horas min perso_dia')
        model.par_P_Di_TH = Param(model.set_turnos, model.set_horas,initialize=dic_P_Di_TH, default=0, doc='disponibilidad turno_hora')
        model.par_P_Hcon_P = Param(model.set_personas,initialize=dic_P_Hcon_P_cluster, default=0, doc='horas contratadas pers_sem')
        model.par_P_Tmax_PD = Param(model.set_personas,model.set_dias,initialize=dic_P_Tmax_PD_cluster, default=default_P_Tmax_PD, doc='maximas comb turno_cons por  pers_dia')


        # Declaracion de variables -------------------------------------------------------------------------------------------------------------------------------------------
        model.var_V_A_PTCD = Var(model.set_personas_consultorios, model.set_turnos, model.set_dias, within=Binary, bounds=(0.0,None), doc='asignacion persona_turno_cons_dia')

        # Declaracion de variables auxiliares --------------------------------------------------------------------------------------------------------------------
        model.var_V_HD_P = Var(model.set_personas, within=NonNegativeReals, bounds=(0.0,None), doc='horas deficit persona_semana')
        model.var_V_A_PD = Var(model.set_personas, model.set_dias, within=Binary, bounds=(0.0,None), doc='asignacion persona_dia')


        # Restricciones -----------------------------------------------------------------------------------------------------

        print('----------------------------------------------------------')
        print('resolviendo cluster residual')
        print('agregando restricciones')
        print(time.strftime("%H:%M:%S"))

        # Restriccion 1: R1 - maximo una combinacion turno-consultorio por persona, dia y hora si la persona esta disponible-------------------------------------------------
        model.con_R1 = ConstraintList(doc='maximo una combinacion turno-consultorio por persona, dia y hora')

        # funcion para calcular consultorios indexados a personas
        def consul_inde_p(p1):
            vector_consul_inde_p=[]
            for i1 in lis_P_Di_PC_cluster:
                if i1[0]==p1:
                    vector_consul_inde_p.append(i1[1])
            return vector_consul_inde_p
        for p in model.set_personas:
            set_consultorios_p=consul_inde_p(p)
            for d in model.set_dias:
                for h in model.set_horas:
                    model.con_R1.add(
                        sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                            for t in model.set_turnos for c in set_consultorios_p) 
                        <=
                        model.par_P_Di_PDH[p,d,h]
                    )
        print('agregada R1')
        print(time.strftime("%H:%M:%S"))

        # Restriccion 2: R2 - maximo una combinacion persona-turno por consultorio, dia y hora si consultorio esta disponible-------------------------------------------------
        model.con_R2 = ConstraintList(doc='maximo una combinacion persona-turno por consultorio, dia y hora')

        # funcion para calcular personas indexadas por consultorio
        def personas_inde_c(c1):
            vector_personas_inde_c=[]
            for i1 in lis_P_Di_PC_cluster:
                if i1[1]==c1:
                    vector_personas_inde_c.append(i1[0])
            return vector_personas_inde_c
        for c in model.set_consultorios:
            set_personas_c=personas_inde_c(c)
            if len(set_personas_c)>0:
                for d in model.set_dias:
                    for h in model.set_horas:
                        model.con_R2.add(
                            sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                for t in model.set_turnos for p in set_personas_c) 
                            <=
                            model.par_P_Di_CDH[c,d,h]
                        )
        print('agregada R2')
        print(time.strftime("%H:%M:%S"))

        # Restriccion 4: R4 - No asignar mas horas de las permitidas por persona y dia - si la persona se activa -------------------------------------------------
        model.con_R4 = ConstraintList(doc='No asignar mas horas de las permitidas por persona y dia')
        for p in model.set_personas:
            set_consultorios_p=consul_inde_p(p)
            for d in model.set_dias:
                if model.par_P_Tmax_PD[p,d]>0:
                    model.con_R4.add(
                            sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                for t in model.set_turnos for c in set_consultorios_p for h in model.set_horas) 
                            <=
                            model.par_P_Hmax_PD[p,d]*model.var_V_A_PD[p,d]
                    )
        print('agregada R4')
        print(time.strftime("%H:%M:%S"))


        # Restriccion 5: R5 - No asignar menos horas de las permitidas por persona y dia - si la persona se activa -------------------------------------------------
        model.con_R5 = ConstraintList(doc='No asignar menos horas de las permitidas por persona y dia')
        for p in model.set_personas:
            set_consultorios_p=consul_inde_p(p)
            for d in model.set_dias:
                if model.par_P_Hmin_PD[p,d]>0:
                    model.con_R5.add(
                            sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                                for t in model.set_turnos for c in set_consultorios_p for h in model.set_horas) 
                            >=
                            model.par_P_Hmin_PD[p,d]*model.var_V_A_PD[p,d]
                        )
        print('agregada R5')
        print(time.strftime("%H:%M:%S"))

        # Restriccion 6: R6 - No asignar mas combinaciones turnos-consultorios permitidos por persona y dia si la persona se activa-------------------------------------------------
        model.con_R6 = ConstraintList(doc='No asignar mas combinaciones turnos-consultorios permitidos por persona')
        for p in model.set_personas:
            set_consultorios_p=consul_inde_p(p)
            for d in model.set_dias:
                model.con_R6.add(
                        sum(model.var_V_A_PTCD[p,c,t,d]
                            for t in model.set_turnos for c in set_consultorios_p) 
                        <=
                        model.par_P_Tmax_PD[p,d]*model.var_V_A_PD[p,d]
                    )
        print('agregada R6')
        print(time.strftime("%H:%M:%S"))


        # Restriccion 7: R7 - Calcular horas deficit por persona a la semana ---------------------------------------
        model.con_R7 = ConstraintList(doc='Calcular horas deficit por persona a la semana')
        for p in model.set_personas:
            set_consultorios_p=consul_inde_p(p)
            model.con_R7.add(
                    sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h]
                        for t in model.set_turnos for c in set_consultorios_p 
                        for h in model.set_horas for d in model.set_dias)
                    + model.var_V_HD_P[p]
                    ==
                    model.par_P_Hcon_P[p]
                )
        print('agregada R7')
        print(time.strftime("%H:%M:%S"))


        # Restriccion 8: R8 - simetria 1: simetria a nivel persona, dia1, dia2 -----------------------------------------
        # garantizar que la persona p, en el dia1 y dia2, trabaje las mismas horas
        if len(simetria_1_cluster)>0:
            model.con_R8 = ConstraintList(doc='simetria 1: simetria a nivel persona, dia1, dia2')
            for sim_1 in simetria_1_cluster:
                p=sim_1[0]
                dia1=sim_1[1]
                dia2=sim_1[2]
                set_consultorios_p=consul_inde_p(p)
                for t in model.set_turnos:
                    model.con_R8.add(sum(model.var_V_A_PTCD[p,c,t,dia1] for c in set_consultorios_p)==
                                     sum(model.var_V_A_PTCD[p,c,t,dia2] for c in set_consultorios_p)
                                    )
            print('agregada R8')
            print(time.strftime("%H:%M:%S"))
            #model.con_R8.pprint()


        # Restriccion 9: R9 - simetria 2: simetria a nivel persona,consultorio, dia1, dia2 -----------------------------------------
        # garantizar que la persona p, en el dia1 y dia2, trabaje las mismas horas y en el mismo consultorio
        if len(simetria_2_cluster)>0:
            model.con_R9 = ConstraintList(doc='simetria 2: simetria a nivel persona,consultorio, dia1, dia2')
            for sim_2 in simetria_2_cluster:
                p=sim_2[0]
                dia1=sim_2[1]
                dia1=sim_2[2]
                set_consultorios_p=consul_inde_p(p)

                for c in set_consultorios_p:
                    for t in model.set_turnos:
                        model.con_R9.add(model.var_V_A_PTCD[p,c,t,dia1]==
                                         model.var_V_A_PTCD[p,c,t,dia2]
                                        )
            print('agregada R9')
            print(time.strftime("%H:%M:%S"))
            #model.con_R9.pprint()      


        # Restriccion 10: R10 - fatiga 10 horas: minimo 10 horas de descanso entre dias consecutivos---------------------
        # garantizar que toda pesona, descanse al menos 10 horas en dias consecutivos

        # funcion para calcular turnos indexados a horas
        def turnos_inde_h(h1):
            vector_turnos_inde_h=[]
            for i1 in dic_P_Di_TH:
                if i1[1]==h1:
                    vector_turnos_inde_h.append(i1[0])
            return vector_turnos_inde_h
        if len(fatiga_10h_cluster)>0:
            model.con_R10 = ConstraintList(doc='fatiga 10 horas: minimo 10 horas de descanso entre dias consecutivos')
            for fat_10 in fatiga_10h_cluster:
                p=fat_10[0]
                dia1=fat_10[1]
                hora1=fat_10[2]
                dia2=fat_10[3]
                hora2=fat_10[4]                
                set_turnos_dis_h1=turnos_inde_h(hora1)
                set_turnos_dis_h2=turnos_inde_h(hora2)
                set_consultorios_p=consul_inde_p(p)

                model.con_R10.add(sum(model.var_V_A_PTCD[p,c1,t1,dia1] for c1 in set_consultorios_p for t1 in set_turnos_dis_h1)+
                                  sum(model.var_V_A_PTCD[p,c2,t2,dia2] for c2 in set_consultorios_p for t2 in set_turnos_dis_h2)<=1
                                 )

            print('agregada R10')
            print(time.strftime("%H:%M:%S"))
            #model.con_R10.pprint() 

        # Restriccion 11: R11 - fatiga semana: maximo 60 horas de trabajo por semana ---------------------
        # garantizar que toda pesona, trabaje maximo 60 horas a la semana

        if len(semanas)>1:
            # funcion para calcular dias indexados a semana
            def dias_inde_sem(s1):
                vector_dias_inde_s=[]
                for i1 in dias_semana:
                    if i1[0]==s1:
                        vector_dias_inde_s.append(i1[1])
                return vector_dias_inde_s

            model.con_R11 = ConstraintList(doc='fatiga semana: maximo 60 horas de trabajo por semana')
            for p in model.set_personas:
                set_consultorios_p=consul_inde_p(p)
                for s in semanas:
                    set_dias_semana_s=dias_inde_sem(s)
                    model.con_R11.add(
                        sum(model.var_V_A_PTCD[p,c,t,d]*model.par_P_Di_TH[t,h] 
                            for t in model.set_turnos for c in set_consultorios_p
                            for h in model.set_horas for d in set_dias_semana_s)
                        <=60
                    )        
            print('agregada R11')
            print(time.strftime("%H:%M:%S"))                    

        # ---------------------- fin restricciones ---------------------------------------------------------               

        # funcion objetivo ----------------------------------------------------------------------------------------------------
        def objective_rule(model):     
            return (sum(model.var_V_HD_P[p] for p in model.set_personas))
        model.objective = Objective(rule=objective_rule, sense=minimize, doc='Objective function')

        # Solucion del modelo -------------------------------------------------------------------------------------------------
        instance = model 
        solver = SolverFactory(solver_name)

        if solver_name=="glpk":
            solver.options["mipgap"] = gap
            solver.options["tmlim"] = 2*Tiempo_maximo_corrida_cluster

        elif solver_name=="cbc":
            solver.options['seconds'] = 2*Tiempo_maximo_corrida_cluster
            solver.options["RatioGap"] = gap
            #solver.options["MaxSolutions"] = 1
            #solver.options['integerTolerance'] = 1E-4


        print('resolviendo el modelo del cluster residual')
        resultados = solver.solve(model)



        # 3. fin modelo de optimizacion para el grupo -----------------------------
        #################################-------------------#########################



        # si se llega a la solucion factible o la optima

        if (resultados.solver.termination_condition=='optimal' or resultados.solver.termination_condition=='feasible'):
            print(' ')   
            print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')            
            print('modelo resuelto----------------------------------------')
            resultados.write()  

            print(time.strftime("%H:%M:%S"))
            grupos_resueltos=grupos_resueltos+1
            # guardar resultados--------------------------------------------
            for p_c in model.set_personas_consultorios:
                for t in model.set_turnos:
                    for d in model.set_dias:
                        if (model.var_V_A_PTCD[p_c,t,d].value>0):
                            resultados1.append([p_c[0],p_c[1],t,d,model.var_V_A_PTCD[p_c,t,d].value])
                            #print(str((p_c,t,d))+str( ' = ')+str(model.var_V_A_PTCD[p_c,t,d].value))


            for p in model.set_personas:
                resultados2.append([p,model.var_V_HD_P[p].value])
                #print(p,model.var_V_HD_P[p].value)
                for d in model.set_dias:
                    resultados3.append([p,d,model.var_V_A_PD[p,d].value])

            for c in model.set_consultorios:
                set_personas_c=personas_inde_c(c)
                for d in model.set_dias:
                    for h in model.set_horas:
                        set_turnos_dis_h=turnos_inde_h(h)
                        cons_act=sum(model.var_V_A_PTCD[p,c,t,d].value for p in set_personas_c for t in set_turnos_dis_h)
                        resultados4.append([c,d,h,cons_act])

        #----------------------------------------------------------------------------------------------------------------

    # imprimir resultados ------------------------------------------------------------------------------------------------
    output_file_name='Solution_'+'solver_'+str(solver_name)+'_TCMax_'+str(default_P_Tmax_PD)+'_NClu_'+str(P_NG)+'_'+'_FaHQ_'+str(factor_HQ)+'_'+ str(filename)
    data1=pd.DataFrame.from_records(resultados1[1:], columns=resultados1[0])
    data2=pd.DataFrame.from_records(resultados2[1:], columns=resultados2[0])
    data2=data2.groupby(['persona'])['horas no programadas'].min().reset_index()
    writer = pd.ExcelWriter(output_file_name)
    data1.to_excel(writer,'Hoja1')
    data2.to_excel(writer,'Hoja2')
    #data3.to_excel(writer,'Hoja3')
    #data4.to_excel(writer,'Hoja4')
    writer.close()


    # horas no programadas antes de cluster residual
    if P_NG>1:
        suma=0
        for i in horas_contr_clus_residual:
            suma+=i[1]
        print(suma)


 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
resolviendo el modelo de clusterizacion---------------------------------
 
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 260
  Number of variables: 593
  Number of nonzeros: 1777
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: feasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of crea

agregada R4
23:47:49
agregada R5
23:47:57
agregada R6
23:47:57
agregada R7
23:48:09
agregada R11
23:48:22
resolviendo el modelo del cluster residual
 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
modelo resuelto----------------------------------------
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 100.0
  Upper bound: 100.0
  Number of objectives: 1
  Number of constraints: 9438
  Number of variables: 142952
  Number of nonzeros: 2042504
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Stati

 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
resolviendo cluster 4 de 4
----------------------------------------------------------
resolviendo cluster 4
agregando restricciones
23:57:52
agregada R1
23:58:04
agregada R2
23:58:17
agregada R4
23:58:27
agregada R5
23:58:37
agregada R6
23:58:37
agregada R7
23:58:49
agregada R11
23:59:02
resolviendo el modelo
 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
modelo resuelto----------------------------------------
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 226.0
  Upper bound: 226.0
  Number of objectives: 1
  Number of constraints: 4977
  Number of varia

agregada R2
00:20:47
agregada R4
00:20:48
agregada R5
00:20:48
agregada R6
00:20:49
agregada R7
00:20:50
agregada R11
00:20:52
resolviendo el modelo
 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
modelo resuelto----------------------------------------
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 52.0
  Upper bound: 52.0
  Number of objectives: 1
  Number of constraints: 2607
  Number of variables: 22726
  Number of nonzeros: 302444
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistic

----------------------------------------------------------
resolviendo cluster residual
agregando restricciones
00:26:54
agregada R1
00:27:00
agregada R2
00:27:06
agregada R4
00:27:07
agregada R5
00:27:08
agregada R6
00:27:09
agregada R7
00:27:15
agregada R11
00:27:21
resolviendo el modelo del cluster residual
 
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
modelo resuelto----------------------------------------
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 105.0
  Upper bound: 105.0
  Number of objectives: 1
  Number of constraints: 5187
  Number of variables: 76938
  Number of nonzeros: 1014148
  Sense: minimize
# -------------------------------------------